In [1]:
from skimage.feature import hog
from scipy.misc import imread
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# from collections import namedtuple
# Point = namedtuple('Point', ['x', 'y'])
# Box = namedtuple('Box', ['top_left', 'bottom_right'])

In [3]:
# def win2blk(i):
#     return list(range(i, i+cells_per_win-1))

# def win2box(i, j):
#     top_left_x = j * pxls_per_cell
#     top_left_y = i * pxls_per_cell
#     bottom_right_x = top_left_x + (cells_per_win*pxls_per_cell)
#     bottom_right_y = top_left_y + (cells_per_win*pxls_per_cell)
#     return Box(top_left=Point(x=top_left_x, y=top_left_y),
#               bottom_right=Point(x=bottom_right_x, y=bottom_right_y))

In [4]:
# img = np.random.normal(loc=128, scale=24, size=(96, 96))
# plt.imshow(img, cmap='gray')
img = imread('/data/vehicle-tracking/test_images/test6.jpg')
img = img.astype(np.float32) / 255
img = img[:, :, 0]

In [5]:
ymax, xmax = img.shape
pxls_per_cell = 8
cells_per_blk = 2
win = 64  # pixels
stride = 16  # pixels

In [6]:
tot_cells_x = xmax // pxls_per_cell
nx_blks = tot_cells_x - cells_per_blk + 1

tot_cells_y = ymax // pxls_per_cell
ny_blks = tot_cells_y - cells_per_blk + 1

print(nx_blks, ny_blks)

159 89


In [7]:
cells_per_win = win // pxls_per_cell
blks_per_win = cells_per_win - cells_per_blk + 1
print(blks_per_win)

7


In [8]:
nx_wins = ((xmax - win) // stride) + 1
print(nx_wins)

77


In [9]:
def win2blk(i):
    start = i * (stride // pxls_per_cell)
    end = start + blks_per_win
    return start, end


def win2pxl(i):
    start = i * stride
    end = start + win
    return start, end

In [10]:
blocks = hog(
    img,
    orientations=9,
    pixels_per_cell=(pxls_per_cell, pxls_per_cell),
    cells_per_block=(cells_per_blk, cells_per_blk),
    transform_sqrt=False,
    feature_vector=False
)
print(blocks.shape)

/home/avilay/anaconda3/envs/dl/lib/python3.6/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


(89, 159, 2, 2, 9)


In [11]:
# Consider window(5, 5)
x_ndx, y_ndx = 5, 5

x_blk = win2blk(x_ndx)
y_blk = win2blk(y_ndx)
print(x_blk, y_blk)

x = win2pxl(x_ndx)
y = win2pxl(y_ndx)
print(x, y)

(10, 17) (10, 17)
(80, 144) (80, 144)


In [12]:
f1 = blocks[10:17, 10:17].ravel()

In [13]:
win_img = img[80:144, 80:144]
print(win_img.shape)
f2 = hog(
    win_img,
    orientations=9,
    pixels_per_cell=(pxls_per_cell, pxls_per_cell),
    cells_per_block=(cells_per_blk, cells_per_blk),
    transform_sqrt=False,
    feature_vector=True
)

(64, 64)


/home/avilay/anaconda3/envs/dl/lib/python3.6/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


In [14]:
print(f1.shape, f2.shape)

(1764,) (1764,)


In [15]:
np.sum(np.logical_not(np.isclose(f1, f2, 1e-3)))

836

In [16]:
f1[:10]

array([ 0.00877308,  0.00177146,  0.        ,  0.        ,  0.0079222 ,
        0.00118097,  0.00485493,  0.00118097,  0.        ,  0.06844871])

In [17]:
f2[:10]

array([ 0.00823423,  0.00176916,  0.        ,  0.        ,  0.00923057,
        0.        ,  0.00335674,  0.        ,  0.        ,  0.06835788])